<a href="https://colab.research.google.com/github/rain9780/excecise/blob/main/chroma_0313.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install langchain-community
! pip install -U pypdf
! pip install -U langchain-google-genai
! pip install -U chromadb
! pip install -U langchain-chroma
! pip install -U langchain-core

In [ ]:
! pip install langchain-core

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import getpass
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
import google.generativeai as genai

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google AI API 키를 입력하세요: ")

google_api_key = os.environ.get("GOOGLE_API_KEY")

google_embeding = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=google_api_key
    )

loader = PyPDFLoader(r"/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
)
docs=text_splitter.split_documents(pages)

db = Chroma.from_documents(docs, google_embeding)
docs = text_splitter.split_documents(pages)

In [ ]:
docs[-1]

Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'page': 13, 'page_label': '14'}, page_content='법제처                                                            14                                                       국가법령정보센터\n대한민국헌법\n제5조 이 헌법시행 당시의 법령과 조약은 이 헌법에 위배되지 아니하는 한 그 효력을 지속한다.\n제6조 이 헌법시행 당시에 이 헌법에 의하여 새로 설치될 기관의 권한에 속하는 직무를 행하고 있는 기관은 이 헌법에\n의하여 새로운 기관이 설치될 때까지 존속하며 그 직무를 행한다.')

In [ ]:
query="제70조의 대통령의 임기는?"
docs = db.similarity_search(query)
print(docs[0])
db.similarity_search_with_score(query)

In [ ]:
#save to disk
db2 = Chroma.from_documents(docs, google_embeding, persist_directory="./Chroma_db")

#load from disk
db3=Chroma(persist_directory="./Chroma_db", embedding_function=google_embeding)
result=db3.similarity_search(query)
print(result[0].page_content)

제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정한 국무위원의 순서로
그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책을 국민투표에 붙일 수
있다.
 
제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강화를 한다.
 
제74조 ①대통령은 헌법과 법률이 정하는 바에 의하여 국군을 통수한다.
②국군의 조직과 편성은 법률로 정한다.
 
제75조 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률을 집행하기 위하여 필요한 사항에 관하여
대통령령을 발할 수 있다.
 
제76조 ①대통령은 내우ㆍ외환ㆍ천재ㆍ지변 또는 중대한 재정ㆍ경제상의 위기에 있어서 국가의 안전보장 또는 공공의


<ipython-input-27-bb29cd640ba7>:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db3=Chroma(persist_directory="./Chroma_db", embedding_function=google_embeding)


In [ ]:
import chromadb
#collection을 저장할 경로 지정
client = chromadb.PersistentClient(path="./collection_example")
client.heartbeat()

1737594396011648708

In [ ]:
from chromadb.utils import embedding_functions
import chromadb.utils.embedding_functions as embedding_functions

embedding_function = embedding_functions.GoogleGenerativeAiEmbeddingFunction(google_api_key)

collection=client.get_or_create_collection(name="korean_law", embedding_function=embedding_function)

In [ ]:
collection

Collection(name=korean_law)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma

loader=PyPDFLoader(r"/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages=loader.load_and_split()

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
)
docs=text_splitter.split_documents(pages)

collection.add(
    ids=[str(i) for i in range(len(docs))],
    documents=[doc.page_content for doc in docs],
    metadatas=[doc.metadata for doc in docs]
)

In [ ]:
Collection=client.get_collection(name="korean_law")

In [ ]:
collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3,
    where={"page":1},
)

In [ ]:
collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3,
    where={"page":{"$gte":5}},
)

In [ ]:
collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3,
    where={"page":1},
    where_document={"$contains":"10"},
)

In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
import os
import getpass

Chroma().delete_collection()

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google AI API 키를 입력하세요: ")
google_api_key = os.environ.get("GOOGLE_API_KEY")

loader = PyPDFLoader(r"/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
)
docs=text_splitter.split_documents(pages)

google_embeding = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=google_api_key
    )
db = Chroma.from_documents(docs, google_embeding)

retreiver = db.as_retriever()
retreiver.invoke("국회의원의 의무")


In [ ]:
result_source=db.similarity_search_with_score("국회의원의 의무")
result_r_source=db.similarity_search_with_relevance_scores("국회의원의 의무")
print("[유사청크 1순위]")
print(result_source[0][0].page_content)
print("\n\n[점수]")
print(result_source[0][1])
print(result_r_source[0][1])

In [ ]:
retreiver=db.as_retriever(search_kwargs={"k":1})
retreiver.get_relevant_documents("국회의원의 의무")

In [ ]:
!pip install --upgrade transformers
!pip install --upgrade sentence-transformers timm

In [ ]:
!pip install langchain-openai

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import os
import getpass





Chroma().delete_collection()

# if "GOOGLE_API_KEY" not in os.environ:
#     os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google AI API 키를 입력하세요: ")
# google_api_key = os.environ.get("GOOGLE_API_KEY")
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API 키를 입력하세요: ")
openai_api_key = os.environ.get("OPENAI_API_KEY")

loader = PyPDFLoader(r"/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
)
docs=text_splitter.split_documents(pages)

# google_embeding = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001",
#     google_api_key=google_api_key
#     )


db = Chroma.from_documents(docs, OpenAIEmbeddings(model='text-embedding-3-small'))

retreiver = db.as_retriever()
retreiver.invoke("국회의원의 의무")

### MMR(Maximal Marginal Relevance) 검색방식

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

loader=PyPDFLoader(r"/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages=loader.load_and_split()

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
)
docs=text_splitter.split_documents(pages)

db = Chroma.from_documents(docs, GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

# retreiver=db.as_retriever(search_type="mmr")
# retreiver.invoke("국회의원의 의무")

#### MultivectorRetriever

In [ ]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

loader=PyPDFLoader(r"/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf")
docs=loader.load_and_split()

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=0,
)
docs=text_splitter.split_documents(docs)

[Document(metadata={'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2024-04-01T21:26:24+09:00', 'moddate': '2024-04-01T21:26:24+09:00', 'source': '/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'total_pages': 14, 'page': 0, 'page_label': '1'}, page_content='법제처                                                            1                                                       국가법령정보센터\n대한민국헌법\n \n대한민국헌법\n[시행 1988. 2. 25.] [헌법 제10호, 1987. 10. 29., 전부개정]\n       전문\n \n유구한 역사와 전통에 빛나는 우리 대한국민은 3ㆍ1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한\n4ㆍ19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의ㆍ인도와 동포애로써 민족의 단\n결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고\n히 하여 정치ㆍ경제ㆍ사회ㆍ문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며\n, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적\n인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다\n짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정\n한다.\n \n1987년 10월

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name="jhgan/ko-sbert-nli"
model_kwargs={'device':'cpu'}
encode_kwargs={'normalize_embeddings':True}
embedding=HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectorstore=Chroma(
    collection_name="full_documents",
    embedding_function=embedding
)

# Decalre layer for saveing upper document 상위 문서 저장 위한 레이어 선언
store=InMemoryStore()
id_key="doc_id"

# Use doc_id as the key value to link upper and lower documents 상위문서와 하위문서를 연결한 키값으로 doc_id 사용

retriever=MultiVectorRetriever(
    vectorstore=vectorstore,
    store=store,
    id_key=id_key
)

# Call the uuid library to assign a unique value to the document id 문서 id로 고유한 값을 지정하기 위하여 uuid 라이브러리 호출

import uuid
doc_ids=[str(uuid.uuid4()) for _ in docs]

#### Long-Context Recorder 없이 유사문서 출력

In [ ]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.document_transformers import LongContextReorder
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import google.generativeai as genai
import os
import getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google AI API 키를 입력하세요: ")

google_api_key = os.environ.get("GOOGLE_API_KEY")

# Declare Korean embedding model
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

texts=[
    "바스켓볼은 훌륭한 스포츠입니다.",
    "플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.",
    "셀틱스는 제가 가장 좋아하는 팀입니다.",
    "이것은 보스턴 셀틱스에 관한 문서입니다.",
    "보스턴 셀틱스는 바스켈볼의 훌륭한 팀입니다.",
    "저는 단순히 영화보러 가는것을 좋아합니다.",
    "보스터 셀틱스가 20점차로 이겼어요.",
    "이것은 그냥 임의의 텍스트입니다.",
    "엘든 링은 지낭 15년 동안 최고의 게임 중 하나입니다.",
    "L. 코넷은 최고의 셀틱스 선수 중 한명입니다.",
    "래리 버드는 상징적인 NBA 선수였습니다.",
]

# Delete existing Chroma collection if it exists
# This prevents potential duplicates when re-running the code
Chroma().delete_collection()

# Chroma Retriever 선언(10개의 유사문서 출력)
retriver= Chroma.from_texts(texts, embedding=embedding).as_retriever(search_kwargs={"k":10})

query="셀틱에 대해 설명해줘"

# 유사도 기준으로 검색 결과 출력
docs=retriver.get_relevant_documents(query)
docs

[Document(id='2689b162-ccc3-4c46-aba0-4ccb0928ead2', metadata={}, page_content='이것은 보스턴 셀틱스에 관한 문서입니다.'),
 Document(id='3c40f68d-2e72-4e48-9bf1-1ccc5aeba0a2', metadata={}, page_content='L. 코넷은 최고의 셀틱스 선수 중 한명입니다.'),
 Document(id='b5756728-3f54-4132-95ee-f8758f4d4833', metadata={}, page_content='보스턴 셀틱스는 바스켈볼의 훌륭한 팀입니다.'),
 Document(id='5c87efc1-b96d-4980-b53e-ad5e2d41e5ac', metadata={}, page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(id='c2330a00-9034-4325-83b3-662bd383bc5f', metadata={}, page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(id='2a83d4de-830a-4cee-b0fe-9626d37cc7c2', metadata={}, page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(id='4a8f8658-0798-41ac-b87f-0f3ac4244ef3', metadata={}, page_content='보스터 셀틱스가 20점차로 이겼어요.'),
 Document(id='1ea512fc-71ff-449d-a00e-a1178cb360ad', metadata={}, page_content='플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.'),
 Document(id='aeac0fe5-399d-4e49-a57a-98a4b3109b9a', metadata={}, page_content='엘든 링은 지낭 15년 동안 최고의 게임 중 하나입니다.'),
 Document(id='40a9493e-d123-42

#### Long-Context Recorder를 활용한 유사문서 출력

In [ ]:
#LongContextRecorder 선언
recorder=LongContextReorder()

#검색된 유사 문서 중 관련도가 높은 문서를 맨 앞과 맨 뒤에 재배치
reordered_docs=recorder.transform_documents(docs)
reordered_docs

[Document(id='3c40f68d-2e72-4e48-9bf1-1ccc5aeba0a2', metadata={}, page_content='L. 코넷은 최고의 셀틱스 선수 중 한명입니다.'),
 Document(id='5c87efc1-b96d-4980-b53e-ad5e2d41e5ac', metadata={}, page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(id='2a83d4de-830a-4cee-b0fe-9626d37cc7c2', metadata={}, page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(id='1ea512fc-71ff-449d-a00e-a1178cb360ad', metadata={}, page_content='플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.'),
 Document(id='40a9493e-d123-42fe-bd9e-36648910eb8f', metadata={}, page_content='래리 버드는 상징적인 NBA 선수였습니다.'),
 Document(id='aeac0fe5-399d-4e49-a57a-98a4b3109b9a', metadata={}, page_content='엘든 링은 지낭 15년 동안 최고의 게임 중 하나입니다.'),
 Document(id='4a8f8658-0798-41ac-b87f-0f3ac4244ef3', metadata={}, page_content='보스터 셀틱스가 20점차로 이겼어요.'),
 Document(id='c2330a00-9034-4325-83b3-662bd383bc5f', metadata={}, page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(id='b5756728-3f54-4132-95ee-f8758f4d4833', metadata={}, page_content='보스턴 셀틱스는 바스켈볼의 훌륭한 팀입니다.'),
 Document(id='2689b162-ccc3-4

#### LCEL(**L**ang**C**hain **E**xpression **L**anguege) 기본 체인 구성

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import GoogleGenerativeAI

#프롬프트 템플릿 설정
prompt=ChatPromptTemplate.from_template("{topic} 내부업무적용 효과에 대해서 설명해줘")

#LLM 호출
llm = GoogleGenerativeAI(model="models/gemini-2.0-pro-exp-02-05")

#출력파서 설정
output_parser=StrOutputParser()

#LCEL로 프롬프트 템플릿-LLM-출력 파서 연결하기
chain=prompt | llm | output_parser

#invoke함수로 chain 실행하기
print(chain.invoke({"topic":"생성형 AI"}))

#### 스트리밍 기능을 쉽게 추가하는 stream()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import GoogleGenerativeAI

#chain 선언
model=GoogleGenerativeAI(model="models/gemini-2.0-pro-exp-02-05")
prompt=ChatPromptTemplate.from_template("{topic} 내부업무적용 효과에 대해서 설명해줘")

chain= prompt | model

#chain의 stream()함수를 통해 스트리밍 기능 추가
for token in chain.stream({"topic":"AI agent"}):
    print(token, end="", flush=True)

#### 여러개의 API를 요청하고 받는 batch()

In [ ]:
import time
model=GoogleGenerativeAI(model="models/gemini-2.0-pro-exp-02-05")
prompt=ChatPromptTemplate.from_template("다음 한글 문장을 일본어로 번역해줘 {sentence}")

chain= prompt | model

results = []
for input_data in [
       {"sentence":"그녀는 매일 아침 책을 읽습니다."},
       {"sentence":"오늘 날씨가 참 좋네요."},
    #    {"sentence":"저녁에 친구들과 영화를 볼 거에요."},
]:
    results.append(chain.invoke(input_data))
    time.sleep(10)  # Introduce a 1-second delay between requests

print(results)

['다음은 "그녀는 매일 아침 책을 읽습니다"를 일본어로 번역한 결과입니다.\n\n* **彼女は毎朝本を読みます。** (Kanojo wa maiasa hon o yomimasu.)\n\n이 번역이 가장 일반적이고 자연스러운 표현입니다.', '오늘 날씨가 참 좋네요.\n\n다음과 같이 번역할 수 있습니다.\n\n* **오늘 날씨가 참 좋네요:**\n    * **今日は本当にいい天気ですね (Kyou wa hontou ni ii tenki desu ne)** - 일반적인 표현입니다.\n    * **今日はとてもいい天気ですね (Kyou wa totemo ii tenki desu ne)** - "정말" 대신 "매우"를 사용한 표현입니다.\n    * **今日は素晴らしい天気ですね (Kyou wa subarashii tenki desu ne)** - "좋다"를 좀 더 격식 있게 표현한 "훌륭하다"를 사용한 표현입니다.\n\n어떤 표현을 사용해도 의미는 잘 전달됩니다. 상황이나 상대방에 따라 적절한 표현을 선택하시면 됩니다.']


#### 1개 문장 번역 수행

In [ ]:
%%time
model=GoogleGenerativeAI(model="models/gemini-2.0-pro-exp-02-05")
prompt=ChatPromptTemplate.from_template("다음 한글 문장을 영어로 번역해줘 {sentence}")

chain= prompt | model

print(chain.invoke({"sentence":"그녀는 매일 아침 책을 읽습니다."}))
print("\n" + "----"*30 + "\n")
print(chain.invoke({"sentence":"오늘 날씨가 참 좋네요."}))

다음은 주어진 한국어 문장을 영어로 번역한 여러 가지 방법입니다:

*   **She reads a book every morning.** (가장 일반적이고 자연스러운 번역)
*   **She reads books every morning.** (매일 여러 권의 책을 읽는다는 뉘앙스)
*   **Every morning, she reads a book.** (문장 구조를 약간 변경)
*    **She has a habit of reading a book every morning.** (매일 아침 책을 읽는 습관이 있다는 것을 강조)

------------------------------------------------------------------------------------------------------------------------

다음은 "오늘 날씨가 참 좋네요"를 영어로 번역한 여러 가지 방법입니다. 상황과 뉘앙스에 따라 적절한 표현을 선택할 수 있습니다.

*   **The weather is really nice today.** (가장 일반적이고 무난한 표현)
*   **It's a beautiful day today.** (아름다운 날씨를 강조)
*   **The weather is lovely today.** (사랑스러운 날씨를 표현)
*   **It's such a nice day out today.** ("밖에" 날씨가 좋다는 것을 포함)
*  **What a beautiful day!** (감탄사 형식)
*    **The weather is great today!**

I recommend using the sentence "The weather is really nice today," because it's simple and natural for all situations.
CPU times: user 53 ms, sys: 4.35 ms, total: 57.3 ms
Wall time: 4.62 s


#### 구글 API 지원모델 확인

In [ ]:
from google import genai

client = genai.Client()

print("List of models that support generateContent:\n")
for m in client.models.list():
    for action in m.supported_actions:
        if action == "generateContent":
            print(m.name)

print("List of models that support embedContent:\n")
for m in client.models.list():
    for action in m.supported_actions:
        if action == "embedContent":
            print(m.name)

### **RAG 시스템 구축 - 기본적인 QA 체인 구성**

#### 필요한 라이브러리 호출 및 API 키 설정

In [ ]:
from langchain import hub
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os
import getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google AI API 키를 입력하세요: ")

google_api_key = os.environ.get("GOOGLE_API_KEY")


#### 문서로드/분활 및 벡터 인베딩

In [ ]:
#헌법 PDF 파일 로드
loader=PyPDFLoader(r"/content/drive/MyDrive/Colab Notebooks/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages=loader.load_and_split()

#PDF 파일을 1000자 청크로 분할
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
docs=text_splitter.split_documents(pages)




27